<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Calibration_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calibration calculation
This skript aims to allow the calculation and generation of a linear regression to allow the quantivication of phthalates in the environement.

nput: The the peak list files (classic structure, header = 6, by thermofisher) are uploaded to the directory titeled *calibration_input*, it is of key importance that the title of the files follows the format: Cal_"conc.nr"_"series.nr"_"identification.nr"_pl.csv Any changes to filename, path or other changes that might impact the file significantly need to be comented and marked extensively.

The following code serves to import the librarys and mount the google drive.

In [ ]:
import numpy as np
from google.colab import drive
import glob
import pandas as pd
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive', force_remount = True)

# Set path to the folder containing the CSV files
path = '/content/drive/My Drive/calibration_input'

This loop aims at importing the various files from the google drive, looping throug the colums and saving them in a new file.

In [ ]:
#First version seems rather unoptimized, hence version 2 seems a lot better, select that one if it seems to work.
# Get a list of all CSV files in the current directory
files = glob.glob('*.csv')

# Loop over each CSV file
for file in files:
    # Read in the CSV file
    df = pd.read_csv(file, header = 6)
    
    # Loop over each row and save a separate data frame for each row
    for i, row in df.iterrows():
        row_df = pd.DataFrame(row).T
        
        # Save the row data frame as a new CSV file
        row_df.to_csv(f'row{i+1}_{file}', index=False)
        
# Get a list of all CSV files with row data
row_files = glob.glob('row*_file*.csv')

# Create an empty list to store the row data frames
row_dfs = []

# Loop over each row file, read it into a data frame, and append it to the list
for file in row_files:
    row_df = pd.read_csv(file)
    row_dfs.append(row_df)

# Concatenate the row data frames along the rows axis
combined_df = pd.concat(row_dfs, axis=0)

# Save the combined data frame to a CSV file
combined_df.to_csv('combined_rows.csv', index=False)


In [ ]:
#version 2 which seems to be more optimized than version one
# Get a list of all CSV files in the current directory
files = glob.glob('Cal_*_pl.csv')

# Loop over each row and save a separate data frame for each row
for i in range(len(pd.read_csv(files[0]).columns)):
    row_dfs = []
    for file in files:
        # Read in the CSV file
        df = pd.read_csv(file)
        
        # Extract the i'th row of the CSV file and append it to the row data frame list
        row = df.iloc[i]
        row_dfs.append(row)
        
    # Concatenate the row data frames along the columns axis and convert to a data frame
    combined_df = pd.concat(row_dfs, axis=1).T
    
    # Save the combined data frame to a CSV file with a name indicating the row number
    combined_df.to_csv('/content/drive/My Drive/calibration_output/row{i+1}.csv', index=False)

The following lines use the previously created row file and take the average and the standard deviation *(or the error)* and add them to two new files.

In [ ]:
# Get a list of all CSV files in the current directory
files = glob.glob('/content/drive/My Drive/calibration_output/row*.csv')
#Loop over each dataframe

for file in files:
  #take the mean of the file and save it in a new data frame
  df_mean = pd.mean(file)
  #save this dataframe in a new csv document
  df_mean.to_csv('/content/drive/My Drive/calibration_output/'file'_mean.csv')
  #take the standard deviation of the file and save it in a new data frame
  df_std = pd.std(file)
  #save this dataframe in a new csv document
  df_std.to_csv('/content/drive/My Drive/calibration_output/'file'_std.csv')